In [1]:
import pandas as pd
url = 'https://nuhuskies.com/sports/womens-ice-hockey/stats/2023-24/connecticut/boxscore/12666'
nu_wih = pd.read_html(url)
len(nu_wih)

14

In [3]:
# Now to build a big loop for all the games
opponents = ['penn-state', 'penn-state', 'liu', 'liu',
             'boston-university', 'boston-university', 'merrimack',
             'st-cloud-state', 'sacred-heart', 'holy-cross', 'holy-cross',
             'connecticut', 'connecticut', 'providence', 'providence',
             'vermont', 'boston-college', 'providence', 'boston-university',
             'yale', 'holy-cross', 'maine', 'maine', 'harvard', 'new-hampshire',
             'boston-university', 'merrimack', 'merrimack', 'new-hampshire',
             'new-hampshire', 'connecticut', 'maine', 'boston-college',
             'boston-college', 'vermont', 'vermont', 'merrimack', 'new-hampshire',
             'connecticut']
game_numbers = [12650, 12653, 12654, 12655, 12656, 12657, 12658,
                12660, 12662, 12663, 12664, 12665, 12666, 12668,
                12671, 12673, 12675, 12677, 12678, 12679, 12680,
                12681, 12682, 12683, 12684, 12685, 12686, 12687,
                12688, 12689, 12690, 12691, 12692, 12693, 12694,
                12695, 13051, 13103, 13104]

In [5]:
# Nested dictionary
full_dict = defaultdict(dict)

# The Loop
for idx in range(len(game_numbers)):
    url = f'https://nuhuskies.com/sports/womens-ice-hockey/stats/2023-24/{opponents[idx]}/boxscore/{game_numbers[idx]}'
    nu_wih = pd.read_html(url)

    # Initialize a dictionary with lists as default values (though lists are not needed here)
    test_dict = defaultdict(list)
    
    # Clean up the team names by removing "Winner " from the "Team" column in nu_wih[0]
    nu_wih[0].iloc[:, 0] = nu_wih[0].iloc[:, 0].str.replace("Winner  ", "")
    
    # Assign home and visit teams to the dictionary
    test_dict['home_team'] = nu_wih[0].loc[1, "Team"]
    test_dict['visit_team'] = nu_wih[0].loc[0, "Team"]
    
    # Extract linescores for home and visit teams
    home_linescore = nu_wih[0].iloc[1, 1:-1]
    visit_linescore = nu_wih[0].iloc[0, 1:-1]
    
    # Assign home linescore values to the dictionary
    for col in home_linescore.index:
        col2 = "home_" + col  # Create a key like "home_1st", "home_2nd", etc.
        test_dict[col2] = home_linescore[col]  # Direct assignment
    
    # Assign visit linescore values to the dictionary
    for col in visit_linescore.index:
        col2 = "visit_" + col  # Create a key like "visit_1st", "visit_2nd", etc.
        test_dict[col2] = visit_linescore[col]  # Direct assignment
    
    # Function to convert the time string (e.g., "02:30") into a timedelta object
    def convert_to_timedelta(time_str):
        minutes, seconds = map(int, time_str.split(':'))
        return timedelta(minutes=minutes, seconds=seconds)
    
    # Function to format timedelta as "MM:SS"
    def format_timedelta(td):
        total_seconds = int(td.total_seconds())
        minutes, seconds = divmod(total_seconds, 60)
        return f"{minutes:02}:{seconds:02}"
    
    # Process home team's power play stats by period from nu_wih[3]
    df_home = nu_wih[3].iloc[:-1, :]  # Exclude last row
    
    if 'Prd' in df_home.columns:
        for period in df_home['Prd'].unique():
            i = int(period[0])  # Extract the period number (e.g., 1st -> 1)
        
            # Filter the data for the specific period
            period_df = df_home[df_home['Prd'] == period]
            
            # Sum shots, opp-shots, and calculate total elapsed power play time
            shots_sum = period_df['Shots'].sum()
            opp_shots_sum = period_df['Opp-Shots'].sum()
            total_elapsed = sum(period_df['Elapsed'].apply(convert_to_timedelta), timedelta())
            total_elapsed_str = format_timedelta(total_elapsed)
            
            # Store the calculated values in the dictionary
            test_dict[f'home_pp_elapsed_{i}'] = total_elapsed_str
            test_dict[f'home_pp_shots_{i}'] = shots_sum
            test_dict[f'home_pp_oppshots_{i}'] = opp_shots_sum
    else:
        test_dict[f'home_pp_elapsed_{i}'] = '00:00'
        test_dict[f'home_pp_shots_{i}'] = 0
        test_dict[f'home_pp_oppshots_{i}'] = 0
        
    # Process visit team's power play stats by period from nu_wih[2]
    df_visit = nu_wih[2].iloc[:-1, :]  # Exclude last row
    
    if 'Prd' in df_visit.columns:
        for period in df_visit['Prd'].unique():
            i = int(period[0])  # Extract the period number
        
            # Filter the data for the specific period
            period_df = df_visit[df_visit['Prd'] == period]
            
            # Sum shots, opp-shots, and calculate total elapsed power play time
            shots_sum = period_df['Shots'].sum()
            opp_shots_sum = period_df['Opp-Shots'].sum()
            total_elapsed = sum(period_df['Elapsed'].apply(convert_to_timedelta), timedelta())
            total_elapsed_str = format_timedelta(total_elapsed)
            
            # Store the calculated values in the dictionary
            test_dict[f'visit_pp_elapsed_{i}'] = total_elapsed_str
            test_dict[f'visit_pp_shots_{i}'] = shots_sum
            test_dict[f'visit_pp_oppshots_{i}'] = opp_shots_sum
    else:
        test_dict[f'visit_pp_elapsed_{i}'] = '00:00'
        test_dict[f'visit_pp_shots_{i}'] = 0
        test_dict[f'visit_pp_oppshots_{i}'] = 0    
    # Process home team performance data from nu_wih[6]
    df_home_perf = nu_wih[6].iloc[-1, :]  # Last row
    
    # Keep relevant columns ('A', 'Shots By Period', 'BLK')
    columns_to_keep = df_home_perf.loc[df_home_perf.index.get_level_values(0).isin(['A', 'Shots By Period', 'BLK'])]
    
    # Store home team performance data in the dictionary
    for col in columns_to_keep.index:
        if col[0] == 'Shots By Period':
            test_dict[f"home_shots_p{col[1]}"] = columns_to_keep[col]
        else:
            test_dict[f'home_{col[0]}'] = columns_to_keep[col]
    
    # Process visit team performance data from nu_wih[5]
    df_visit_perf = nu_wih[5].iloc[-1, :]  # Last row
    
    # Keep relevant columns ('A', 'Shots By Period', 'BLK')
    columns_to_keep = df_visit_perf.loc[df_visit_perf.index.get_level_values(0).isin(['A', 'Shots By Period', 'BLK'])]
    
    # Store visit team performance data in the dictionary
    for col in columns_to_keep.index:
        if col[0] == 'Shots By Period':
            test_dict[f"visit_shots_p{col[1]}"] = columns_to_keep[col]
        else:
            test_dict[f'visit_{col[0]}'] = columns_to_keep[col]
    
    # Store faceoff wins for home and visit teams
    test_dict['home_faceoff_w'] = nu_wih[9].iloc[-1, -1]  # Last row, last column
    test_dict['visit_faceoff_w'] = nu_wih[9].iloc[-1, -2]  # Last row, second-to-last column

    print(f'Index {idx} worked, moving to {idx+1}')
    full_dict[f'game_{idx}'] = test_dict

Index 0 worked, moving to 1
Index 1 worked, moving to 2
Index 2 worked, moving to 3
Index 3 worked, moving to 4
Index 4 worked, moving to 5
Index 5 worked, moving to 6
Index 6 worked, moving to 7
Index 7 worked, moving to 8
Index 8 worked, moving to 9
Index 9 worked, moving to 10
Index 10 worked, moving to 11
Index 11 worked, moving to 12
Index 12 worked, moving to 13
Index 13 worked, moving to 14
Index 14 worked, moving to 15
Index 15 worked, moving to 16
Index 16 worked, moving to 17
Index 17 worked, moving to 18
Index 18 worked, moving to 19
Index 19 worked, moving to 20
Index 20 worked, moving to 21
Index 21 worked, moving to 22
Index 22 worked, moving to 23
Index 23 worked, moving to 24
Index 24 worked, moving to 25
Index 25 worked, moving to 26
Index 26 worked, moving to 27
Index 27 worked, moving to 28
Index 28 worked, moving to 29
Index 29 worked, moving to 30
Index 30 worked, moving to 31
Index 31 worked, moving to 32
Index 32 worked, moving to 33
Index 33 worked, moving to 34

In [6]:
full_dict

defaultdict(dict,
            {'game_0': defaultdict(list,
                         {'home_team': 'Northeastern',
                          'visit_team': 'Penn St.',
                          'home_1': 1,
                          'home_2': 0,
                          'home_3': 0,
                          'visit_1': 0,
                          'visit_2': 0,
                          'visit_3': 0,
                          'home_pp_elapsed_1': '04:05',
                          'home_pp_shots_1': 4,
                          'home_pp_oppshots_1': 0,
                          'home_pp_elapsed_2': '02:00',
                          'home_pp_shots_2': 1,
                          'home_pp_oppshots_2': 0,
                          'home_pp_elapsed_3': '00:40',
                          'home_pp_shots_3': 0,
                          'home_pp_oppshots_3': 1,
                          'visit_pp_elapsed_1': '02:00',
                          'visit_pp_shots_1': 2,
                          

In [7]:
# Convert the nested dictionary into a DataFrame
df_2023_24 = pd.DataFrame.from_dict(full_dict, orient='index')

# Display the DataFrame
df_2023_24


,home_team,visit_team,home_1,home_2,home_3,visit_1,visit_2,visit_3,home_pp_elapsed_1,home_pp_shots_1,...,visit_OT 1,visit_pp_elapsed_4,visit_pp_shots_4,visit_pp_oppshots_4,home_shots_p4,visit_shots_p4,home_OT 2,visit_OT 2,home_shots_p5,visit_shots_p5
game_0,Northeastern,Penn St.,1,0,0,0,0,0,04:05,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_1,Northeastern,Penn St.,1,0,0,0,0,2,02:00,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_2,LIU,Northeastern,0,0,0,2,1,3,02:00,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_3,LIU,Northeastern,0,0,0,0,0,2,04:00,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_4,Boston University,Northeastern,1,0,0,1,1,2,02:00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_5,Northeastern,Boston U.,0,0,2,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_6,Merrimack,Northeastern,0,1,0,0,0,0,02:00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_7,Northeastern,St. Cloud St.,0,0,0,0,0,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_8,Sacred Heart,Northeastern,0,0,1,3,1,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_9,Northeastern,Holy Cross,0,0,0,0,0,0,02:00,7.0,...,1.0,01:02,3.0,0.0,1.0,3.0,NaN,NaN,NaN,NaN
